# Docker CLI


## Docker Pull and Run

To test your docker installation

In [ ]:
docker run hello-world

Let's look at another basic image called _busybox_. Run `busybox` docker image and print a message to the screen:

In [ ]:
# pull down the busybox image locally
docker pull busybox

# run the busybox image. nothing happens! 
docker run busybox

# print a basic hello message!
docker run busybox echo "hello from busybox"

# list the home directory of busybox
docker run busybox ls /

# using additional run options:
# add a --rm option to remove the running container from docker immediately after run
docker run --rm busybox echo "Wish you were here... swimming in a fish bowl year after year"

When you use docker pull or run, docker downloads a copy of the image from docker hub onto your local machine. To see a list of your local images you can use the `docker images` command:

In [ ]:
# see a list of local images
docker images


REPOSITORY                  TAG           IMAGE ID       CREATED        SIZE
hello-world                 latest        feb5d9fea6a5   5 weeks ago    13.3kB
busybox                     latest        388056c9a683   6 months ago   1.23MB


# run images with specific tags (versions)
# ----------------------------------------
# run the latest version (currently this is 3.10)
docker run python:latest python3 -V

# run python 3.9
docker run python:3.9 python3 -V

## Docker ps and rm

1. List running and exited docker containers. 
1. Remove dangling containers
1. Remove images
1. Housekeeping with _'system prune'_

In [ ]:
# show a list of running containers
# it's ok if you don't see anything! That just means there's no actively running containers
docker ps 

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES

# show a list all containers including exited containers
docker ps -a


CONTAINER ID   IMAGE         COMMAND                  CREATED          STATUS                      PORTS     NAMES
3b88d917feda   busybox       "echo 'hello from bu…"   7 seconds ago    Exited (0) 7 seconds ago              musing_lewin
b11b28aa818e   hello-world   "/hello"                 20 seconds ago   Exited (0) 20 seconds ago             upbeat_bassi
2ff40ce83e43   busybox       "ls /"                   26 seconds ago   Exited (0) 25 seconds ago             interesting_jepsen


# remove a container by ID
docker rm -vf 3b88d917feda

# remove a container by NAME
docker rm -vf upbeat_bassi

# remove an image by <name>:<tag>
docker rmi hello-world:latest

# big housekeeping command
docker system prune -f

## Docker run with Terminal Access

When running docker containers with `docker run` you can use the `-it` option to gain terminal access inside the running container. _`-it`_ is short for _--interactive_ _--terminal_.

In [ ]:
# pull the latest ubuntu image (currently ubuntu 20.04 focal)
docker pull ubuntu:latest

# run docker with -it
# pay attention, this command also starts a /bin/bash session upon docker run entry
docker run -it --rm ubuntu:latest /bin/bash
```

Now, you are inside the ubuntu container terminal. You can run any ubuntu command inside the container and exit:

```bash
> ls
bin  boot  dev  etc  home  lib  lib32  lib64  libx32  media  mnt  opt  proc  root  run  sbin  srv  sys  tmp  usr  var

> uname -a
Linux e5af9e019b09 5.11.0-38-generic #42~20.04.1-Ubuntu SMP Tue Sep 28 20:41:07 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux

> cat /etc/os-release 
NAME="Ubuntu"
VERSION="20.04.2 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.2 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal

> exit
exit

## Detached containers and Docker Exec

Another feature of _docker run_ is running containers in detached mode or in background. Containers in detached mode keep running in the background. This is very useful for containers such as web servers which need to continuously run in the background waiting for http requests. Users can later use `docker exec` to attach to a running container and gain terminal access. 

The following example shows:

1. Running an ubuntu bash session in detached mode
1. Listing containers to see the container continues to run in the background
1. Using _'docker exec'_ to gain terminal access to the container
1. Killing and removing the container from the background

In [ ]:
# a little housekeeping first, remove all dangling containers for a clean slate
docker system prune -f

# start a ubuntu bash session in detached mode
#   -d: tells docker to run in the container in the background (detached)
#   -i: tells docker to keep an --interactive interface open to the container
#   --name: names this container my-ubuntu. this is a unique name which should not be used with any other running container
docker run -d -i --name my-ubuntu ubuntu:latest /bin/bash

# list running containers 
# see that my-ubuntu container is still running
docker ps

CONTAINER ID   IMAGE           COMMAND       CREATED         STATUS        PORTS     NAMES
b71672c8e9a2   ubuntu:latest   "/bin/bash"   3 seconds ago   Up 1 second             my-ubuntu

# docker exec runs a command on a running container
#   ie: here we cat the os-release info file
docker exec my-ubuntu cat /etc/os-release

# you can also use exec to run a command and attach to its terminal 
# ie: here we run a new bash session with interactive terminal access
docker exec -it my-ubuntu /bin/bash

# now you're inside the image
> uname -a
Linux e5af9e019b09 5.11.0-38-generic #42~20.04.1-Ubuntu SMP Tue Sep 28 20:41:07 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux

> exit

# exit the terminal with the exit command above.
# Now, you're back at your hots machine

# kill/remove the running container
docker rm -vf my-ubuntu


## Docker Volumes

One of the advantages of docker containers is that they are like a **_gold fish_**; meaning that they don't remember anything! Once you exit a container, everything in it is lost forever. This is actually very helpful to using them as a sandbox. You can start containers, test anything, and don't worry if you're going to break anything. Once you exit the container everything is lost and you can start a new container for your next test!

But what if you want to persist files?

You can persist files in between docker runs by one of two ways:

1. Mount a directory on the host machine to a directory inside the container
1. Use docker volumes which persist in between runs and can be attached to containers

Both these two options use the `-v` option of `docker run` and are explained below

### Attaching host dirs to containers

Now, we're going to:

1. Use the docker run `-v` option to mount the data/ directory to a container
1. Once a directory is mounted, it can be used as a directory inside the container
1. Add line to _names.txt_ file inside the container
1. Exit the container and observe our changes are written to our host dir

In [ ]:
# pay attention to the content of your names.txt file on your host machine\
cat data/names.txt

# run a container and attach the local data/ directory under container's app-data dir
docker run -di --name ubuntu-vol-1 -v $(pwd)/data:/app-data ubuntu /bin/bash

# attach to the container and add a line to names.txt under app-data
docker exec -it ubuntu-vol-1 /bin/bash


# these command are ran inside the container terminal
> ls /app-data
> echo "sweet_horizon" >> /app-data/names.txt
> exit


# you've exited the container terminal now

# examine your names.txt on host machine. see that the new line is added to the end
cat data/names.txt

# remove the image
docker rm -vf ubuntu-vol-1

### Using Docker Volumes

In addition to using local host dirs, you can use docker volumes as the built-in docker feature to persist storage dirs on container. Similar to host dirs, docker volumes are attached to containers using the `-v` option and they persist their data after the container is exited. The volume can later be attached to another container. 

Here we will:

1. Create a docker volume that persists beyond a single docker container
1. Run a container and attached the volume to it
1. Modify files inside the volume
1. Exit the container and run another container with the same volume attached
1. See you changes from the previous run (container)

In [ ]:
# create a new volume
docker volume create ub1-data

# list exiting volumes
docker volume ls
DRIVER    VOLUME NAME
local     ub1-data

# run a container and attach this volume
docker run -it --rm -v ub1-data:/app-data ubuntu /bin/bash

# add a file into our volume (this is running inside the container)
> echo "a data file that persists!" > /app-data/file.txt
> exit

# see that your container has completely exited
docker ps -a

# run another container and attach the same volume
docker run -it --rm -v ub1-data:/app-data ubuntu /bin/bash

# run these commands inside the docker container
> cat /app-data/file.txt
> exit

# now delete the volume 
docker volume rm ub1-data

## Working with Environment Variables

Another way to pass information to a container is via environment variable. You set and pass in environment variables by using the docker run `-e` option. This is commonly used to pass in configuration information to a container.

For example: the [MariaDB](https://hub.docker.com/_/mariadb) docker image uses an environment variable called _MARIADB_ROOT_PASSWORD_ to set the root password for the database. 

In example below, we will:

1. Start a container with a custom print message as an environment variable
1. Enter the container terminal and examine the value of our variable


In [ ]:
# set a custom environment variable
docker run -it --rm -e PRINT_MSG='So, so you think you can tell... heaven from hell?' ubuntu /bin/bash

# inside the container terminal
> echo $PRINT_MSG
> exit

## Exposing Ports

Many containers need to expose their ports to the host machine or other containers. This allows containers to communicate with each other or be accessible by the host machine. 

For example a webserver image (such as _nginx_) needs to expose its default http port 80 to be accessible via a browser on the host machine. 

Docker run uses the `-p` option to expose ports from a container onto the host. The syntax is `-p <port_on_host>:<post_on_container>`.

In the example below, we will:

1. Pull down and use the _[nginx](https://hub.docker.com/_/nginx)_ webserver image
1. Attached our local /data dir to be hosted as the webserver root folder. This folder contains a _index.html_ file that will be hosted.
1. Use the `-p` option to map the container http port (80) onto our host port 8080
1. Access the exposed port via a browser on our host machine

In [ ]:
# pull nginx, a static web server
docker pull nginx

# run a server exposing port 80 of container onto port 8080 of the hots
docker run -d --name nginx-server -v $(pwd)/data:/usr/share/nginx/html:ro -p 8080:80 nginx

Open a browser and go to [http://localhost:8080/](http://localhost:8080/).

You can also get to the same page using the docker network:

```bash
docker inspect nginx-server | grep IPAddress
            "SecondaryIPAddresses": null,
            "IPAddress": "172.17.0.2",
                    "IPAddress": "172.17.0.2",
```

Our container IP address is: _172.17.0.2_. We can view the same page by going to `http://172.17.0.2:80` or just `http://172.17.0.2` since port 80 is the default port.

This requires more understanding of docker networking.



## Exercise

Using the [MariaDB](https://hub.docker.com/_/mariadb) docker hub documentation:

1. Start a mariadb container in detached mode
1. Set its root password to 'lucid_dreams'
1. Expose mariadb port 3306 onto the host
1. Create a docker volume called _db-files_ and attached to where mariadb stores its database files (read teh docs)
1. Connect to the container using _docker exec_ and _mysql_ command line
1. Connect to the container using VSCode MySQL extension 
1. Kill and remove the container and its volume